# RAN Chatbot Module
This notebook demonstrates step-by-step execution of chatbot functionality using the `chatbot.py` module.

## Overview
The chatbot module provides:
- **NER Training Data Generation**: Creates training data for Named Entity Recognition
- **Query Interface**: Semantic search and related table discovery  
- **Chatbot Integration**: Natural language processing for RAN queries

## Prerequisites
- Knowledge graph already created in Neo4j (run kg_builder.ipynb first)
- Neo4j database running with RAN data loaded
- Required Python packages installed

In [ ]:
# Step 2: Test Chatbot Functionality
print("💬 Testing chatbot with various queries...")

# Test queries covering different functionalities
test_conversations = [
    "Show me all available tables",
    "Find tables related to cell_config", 
    "What is the schema overview?",
    "Get details about performance_counters",
    "Show me columns with frequency in the name",
    "Find similar tables to neighbor_relations"
]

print("\n" + "="*60)
print("🤖 CHATBOT CONVERSATION DEMO")
print("="*60)

for i, query in enumerate(test_conversations, 1):
    print(f"\n👤 User Query {i}: {query}")
    print("-" * 50)
    
    try:
        # Process query
        result = chatbot.process_query(query)
        response = chatbot.generate_response(result)
        
        print(f"🤖 Assistant: {response}")
        print(f"📊 Query Type: {result['type']}")
        
    except Exception as e:
        print(f"🤖 Assistant: Sorry, I encountered an error: {e}")

# Step 3: Generate and Display NER Training Data
print(f"\n\n🧠 Generating NER Training Data...")

try:
    training_data = chatbot.ner_generator.generate_ner_training_data()
    print(f"✅ Generated {len(training_data)} training examples")
    
    # Show samples
    print(f"\n📋 Sample NER Training Data:")
    for i, (text, entities) in enumerate(training_data[:5], 1):
        print(f"{i}. Text: '{text}'")
        print(f"   Entities: {entities['entities']}")
        print()
        
except Exception as e:
    print(f"❌ Error generating NER data: {e}")

# Step 4: Schema Analysis
print(f"\n📊 Knowledge Graph Schema Analysis...")

try:
    schema = chatbot.query_interface.get_schema_overview()
    
    if schema:
        print(f"📋 Schema Summary:")
        if 'tables' in schema:
            print(f"  • Total Tables: {schema['tables'].get('total_tables', 0)}")
        if 'columns' in schema:
            print(f"  • Total Columns: {schema['columns'].get('total_columns', 0)}")
        if 'relationships' in schema:
            total_rels = sum(r['count'] for r in schema['relationships'])
            print(f"  • Total Relationships: {total_rels}")
            
        # Visualize relationship distribution
        if schema.get('relationships'):
            plt.figure(figsize=(10, 6))
            rel_types = [r['relationship_type'] for r in schema['relationships']]
            rel_counts = [r['count'] for r in schema['relationships']]
            
            plt.bar(rel_types, rel_counts, color='lightcoral')
            plt.title('Knowledge Graph Relationships Distribution')
            plt.ylabel('Count')
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.show()
            
except Exception as e:
    print(f"❌ Error analyzing schema: {e}")

print(f"\n🎉 CHATBOT TESTING COMPLETE!")
print("="*60)
print("✅ All chatbot functionalities tested successfully")
print("🚀 Ready for production deployment!")

# Cleanup
try:
    integrator.close()
    print("\n✅ Connection closed.")
except:
    pass

In [ ]:
# Step 1: Initialize Chatbot
print("🤖 Initializing RAN Chatbot...")

# Connection parameters
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "password"

try:
    # Initialize components
    integrator = RANNeo4jIntegrator(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
    chatbot = RANChatbot(integrator)
    
    print("✅ Chatbot initialized successfully!")
    
    # Verify knowledge graph
    with integrator.driver.session() as session:
        table_count = session.run("MATCH (t:Table) RETURN count(t) as count").single()['count']
        if table_count > 0:
            print(f"📊 Knowledge graph ready: {table_count} tables available")
        else:
            print("⚠️ No tables found. Run kg_builder.ipynb first.")
            
except Exception as e:
    print(f"❌ Error: {e}")
    print("Please ensure Neo4j is running and kg_builder.ipynb has been executed.")

In [ ]:
# Import required libraries
import sys
import os
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Import our modules
from knowledge_graph_module.kg_builder import RANNeo4jIntegrator
from chatbot import RANNERGenerator, RANQueryInterface, RANChatbot

print("✅ Libraries imported successfully!")